# Modeling code - SSD (Single Shot Multibox Detector)

<b>Table of Contents:</b>
- Configuring modeling pipeline
- Model training using transferred learning
- Exporting a trained inference graph
- Evaluation of the model on train and test data

## Installing packages - check for GPU

In [1]:
# ! pip install --upgrade numpy==1.17.4

In [2]:
# ! pip install --upgrade tensorflow==1.15

In [3]:
import tensorflow
tensorflow.__version__

'1.15.0'

In [4]:
import numpy
numpy.__version__

'1.17.4'

## Model configurations

In [5]:
# Number of training steps.
num_steps = 40000  # 200000

# Number of evaluation steps.
num_eval_steps = 50

# Model configuration
# model_name : name of the model (with different base convnet model architecture)
# Currently written the code for 4 types of configurations:
# 1. ssd_mobilenet_v2
# 2. ssd_inception_v2
# 3. ssd_mobilenet_v3
# 4. Any custom ssd model - only when training was not complete
# As required, mention the name of the model and the model file name along with the respective config file name
# Mention the model to be used in the selected model section
# Any other hyperparameters to be tuned can be mentioned in the MODEL_CONFIG and then stored in a variable in the later part
MODELS_CONFIG = {
    'ssd_mobilenet_v2': {
        'model_name': 'ssd_mobilenet_v2_coco_2018_03_29',
        'pipeline_file': 'ssd_mobilenet_v2_coco.config',
        'batch_size': 24
    },
    'ssd_inception_v2': {
        'model_name': 'ssd_inception_v2_coco_2018_01_28',
        'pipeline_file': 'ssd_inception_v2_coco.config',
        'batch_size': 24
    },
    'ssd_mobilenet_v3': {
        'model_name': 'ssd_mobilenet_v3_large_coco_2019_08_14',
        'pipeline_file': 'ssdlite_mobilenet_v3_large_320x320_coco.config',
        'batch_size': 24
    },
    'ssd_custom_model': {
        'model_name': 'fine_tuned_model_v1',
        'pipeline_file': None,
        'batch_size': 6
    }
}

# Pick the model you want to use
# Select a model in `MODELS_CONFIG`.
selected_model = 'ssd_inception_v2'

# Name of the object detection model to use.
MODEL = MODELS_CONFIG[selected_model]['model_name']

# Name of the pipline file in tensorflow object detection API.
pipeline_file = MODELS_CONFIG[selected_model]['pipeline_file']

# Training batch size fits in Colabe's Tesla K80 GPU memory for selected model.
batch_size = MODELS_CONFIG[selected_model]['batch_size']
# Add any other configurations here after mentioning in the MODEL_CONFIG part

## Installing Tensorflow Object detection API

In [6]:
!ls

Custom_Object_Detector_SSD_v6_(2)_(1)_(3)_(1) (1).ipynb
Custom_Object_Detector_SSD_v7.ipynb
models
nohup.out
Object_Detection_Inference_SSD_v5.1.2.ipynb
object-detection.pbtxt
ssd_inception_v2_coco.config
test.record
train.record
val.record


In [7]:
%cd ~/PNG/Mukesh/Tensorflow/models/research

/home/gpuaffine/PNG/Mukesh/Tensorflow/models/research


In [8]:
import os
os.environ['PYTHONPATH'] = ':/home/gpuaffine/PNG/Mukesh/Tensorflow/models/research:/home/gpuaffine/PNG/Mukesh/Tensorflow/models/research/slim/'

In [9]:
!echo $PYTHONPATH

:/home/gpuaffine/PNG/Mukesh/Tensorflow/models/research:/home/gpuaffine/PNG/Mukesh/Tensorflow/models/research/slim/


In [10]:
!python object_detection/builders/model_builder_test.py

W0429 15:48:48.308618 140596829488896 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Running tests under Python 3.7.4: /home/gpuaffine/anaconda3/envs/tf/bin/python
[ RUN      ] ModelBuilderTest.test_create_experimental_model
[       OK ] ModelBuilderTest.test_create_experimental_model
[ RUN      ] ModelBuilderTest.test_create_faster_rcnn_model_from_config_with_example_miner
[       OK ] ModelBuilderTest.test_create_faster_rcnn_model_from_config_with_example_miner
[ RUN      ] ModelBuilderTest.test_create_faster_rcnn_models_from_config_faster_rcnn_with_matmul
[       OK ] ModelBuilderTest.test_create_faster_rcnn_models_from_config_faster_rcnn_with_matmul
[

In [11]:
#------main part of code

## Path of the test, train and label map data

In [12]:
# Mention the name of the validation and train data (tfrecords) and the labels .pbtxt file
val_record_fname = '/home/gpuaffine/PNG/Dristi/val.record' # validation data
test_record_fname = '/home/gpuaffine/PNG/Dristi/test.record' # test data
train_record_fname = '/home/gpuaffine/PNG/Dristi/train.record' # train data
label_map_pbtxt_fname = '/home/gpuaffine/PNG/Dristi/object-detection.pbtxt' # labels file

In [13]:
# show the present working directory - should be research folder
!pwd


/home/gpuaffine/PNG/Mukesh/Tensorflow/models/research


In [14]:
# Downloading the base pre-trained model from the options mentioned in MODEL_CONFIG
# If we are going to use custom model then the model name would be set to the name of the checkpoint directory
import os
import shutil
import glob
import urllib.request
import tarfile

MODEL_FILE = MODEL + '.tar.gz'
DEST_DIR = '/home/gpuaffine/PNG/Mukesh/Tensorflow/models/research/pretrained_model'
if selected_model.startswith("ssd_custom_model"):
    DOWNLOAD_BASE = '/home/gpuaffine/PNG/Dristi/'
    cmd = 'cp '+DOWNLOAD_BASE + MODEL_FILE+' .'
    os.system(cmd) 
else :
    DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'
    if not (os.path.exists(MODEL_FILE)):
        urllib.request.urlretrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)

tar = tarfile.open(MODEL_FILE)
tar.extractall()
tar.close()

os.remove(MODEL_FILE)
if (os.path.exists(DEST_DIR)):
    shutil.rmtree(DEST_DIR)
os.rename(MODEL, DEST_DIR)

In [15]:
# Checking the pre-trained model folder
!echo {DEST_DIR}
!ls -alh {DEST_DIR}

/home/gpuaffine/PNG/Mukesh/Tensorflow/models/research/pretrained_model
total 197M
drwxr-xr-x  3 gpuaffine gpuaffine 4.0K Feb  2  2018 .
drwxrwxr-x 70 gpuaffine gpuaffine 4.0K Apr 29 15:50 ..
-rw-r--r--  1 gpuaffine gpuaffine   77 Feb  2  2018 checkpoint
-rw-r--r--  1 gpuaffine gpuaffine  98M Feb  2  2018 frozen_inference_graph.pb
-rw-r--r--  1 gpuaffine gpuaffine  96M Feb  2  2018 model.ckpt.data-00000-of-00001
-rw-r--r--  1 gpuaffine gpuaffine  18K Feb  2  2018 model.ckpt.index
-rw-r--r--  1 gpuaffine gpuaffine 3.5M Feb  2  2018 model.ckpt.meta
-rw-r--r--  1 gpuaffine gpuaffine 4.0K Feb  2  2018 pipeline.config
drwxr-xr-x  3 gpuaffine gpuaffine 4.0K Feb  2  2018 saved_model


In [16]:
# model finetuning checkpoint - from where the model would start learning
fine_tune_checkpoint = os.path.join(DEST_DIR, "model.ckpt")
fine_tune_checkpoint

'/home/gpuaffine/PNG/Mukesh/Tensorflow/models/research/pretrained_model/model.ckpt'

## Configuring a Training Pipeline

In [17]:
# Loading the model config file
import os
if not pipeline_file:
    pipeline_fname = os.path.join(DEST_DIR, 'pipeline.config')
else:
    pipeline_fname = os.path.join('/home/gpuaffine/PNG/Mukesh/Tensorflow/models/research/object_detection/samples/configs/',\
                                  pipeline_file)
    cmd = 'cp '+pipeline_fname+' .'
    os.system(cmd)
    pipeline_fname = "./"+pipeline_file

assert os.path.isfile(pipeline_fname), '`{}` not exist'.format(pipeline_fname)

In [18]:
# Function to get the total number of classes for classification
def get_num_classes(pbtxt_fname):
    from object_detection.utils import label_map_util
    label_map = label_map_util.load_labelmap(pbtxt_fname)
    categories = label_map_util.convert_label_map_to_categories(
        label_map, max_num_classes=90, use_display_name=True)
    category_index = label_map_util.create_category_index(categories)
    return len(category_index.keys())

In [19]:
# Modifying the the content of the model config file - re-configuration
import re

num_classes = get_num_classes(label_map_pbtxt_fname)
num_examples = 160
with open(pipeline_fname) as f:
    s = f.read()
with open(pipeline_fname, 'w') as f:
    
    # fine_tune_checkpoint
    s = re.sub('fine_tune_checkpoint: ".*?"',
               'fine_tune_checkpoint: "{}"'.format(fine_tune_checkpoint), s)
    
    # tfrecord files train and validation.
    s = re.sub(
        '(input_path: ".*?)(train.record)(.*?")', 'input_path: "{}"'.format(train_record_fname), s)
    s = re.sub(
        '(input_path: ".*?)(val.record)(.*?")', 'input_path: "{}"'.format(val_record_fname), s)

    # label_map_path
    s = re.sub(
        'label_map_path: ".*?"', 'label_map_path: "{}"'.format(label_map_pbtxt_fname), s)

    # Set training batch_size.
    s = re.sub('batch_size: [0-9]+',
               'batch_size: {}'.format(batch_size), s)

    # Set training steps, num_steps
    s = re.sub('num_steps: [0-9]+',
               'num_steps: {}'.format(num_steps), s)
    
    # Set number of classes num_classes.
    s = re.sub('num_classes: [0-9]+',
               'num_classes: {}'.format(num_classes), s)
    
    # Set number of examples.
    s = re.sub('num_examples: [0-9]+',
               'num_examples: {}'.format(num_examples), s)
    f.write(s)

W0429 15:50:21.916871 140070318745344 module_wrapper.py:139] From /home/gpuaffine/PNG/Mukesh/Tensorflow/models/research/object_detection/utils/label_map_util.py:138: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.



In [20]:
# Displaying the content of the model config file
!cat {pipeline_fname}

# SSD with Inception v2 configuration for MSCOCO Dataset.
# Users should configure the fine_tune_checkpoint field in the train config as
# well as the label_map_path and input_path fields in the train_input_reader and
# eval_input_reader. Search for "PATH_TO_BE_CONFIGURED" to find the fields that
# should be configured.

model {
  ssd {
    num_classes: 5
    box_coder {
      faster_rcnn_box_coder {
        y_scale: 10.0
        x_scale: 10.0
        height_scale: 5.0
        width_scale: 5.0
      }
    }
    matcher {
      argmax_matcher {
        matched_threshold: 0.5
        unmatched_threshold: 0.5
        ignore_thresholds: false
        negatives_lower_than_unmatched: true
        force_match_for_each_row: true
      }
    }
    similarity_calculator {
      iou_similarity {
      }
    }
    anchor_generator {
      ssd_anchor_generator {
        num_layers: 6
        min_scale: 0.2
        max_scale: 0.95
        aspect_ratios: 1.0
        aspect_ratios: 2.0
        aspect_

Editing the configuration file to add the path for the TFRecords files, pbtxt,batch_size,num_steps,num_classes.
Any image augmentation, hyperparemeter tunning (drop out, batch normalization... etc) would be editted here

In [21]:
%%writefile {pipeline_fname}
# SSD with Inception v2 configuration for MSCOCO Dataset.
# Users should configure the fine_tune_checkpoint field in the train config as
# well as the label_map_path and input_path fields in the train_input_reader and
# eval_input_reader. Search for "PATH_TO_BE_CONFIGURED" to find the fields that
# should be configured.

model {
  ssd {
    num_classes: 5
    box_coder {
      faster_rcnn_box_coder {
        y_scale: 10.0
        x_scale: 10.0
        height_scale: 5.0
        width_scale: 5.0
      }
    }
    matcher {
      argmax_matcher {
        matched_threshold: 0.5
        unmatched_threshold: 0.5
        ignore_thresholds: false
        negatives_lower_than_unmatched: true
        force_match_for_each_row: true
      }
    }
    similarity_calculator {
      iou_similarity {
      }
    }
    anchor_generator {
      ssd_anchor_generator {
        num_layers: 6
        min_scale: 0.2
        max_scale: 0.95
        aspect_ratios: 1.0
        aspect_ratios: 2.0
        aspect_ratios: 0.5
        aspect_ratios: 3.0
        aspect_ratios: 0.3333
        reduce_boxes_in_lowest_layer: true
      }
    }
    image_resizer {
      fixed_shape_resizer {
        height: 300
        width: 300
      }
    }
    box_predictor {
      convolutional_box_predictor {
        min_depth: 0
        max_depth: 0
        num_layers_before_predictor: 0
        use_dropout: false
        dropout_keep_probability: 0.8
        kernel_size: 3
        box_code_size: 4
        apply_sigmoid_to_scores: false
        conv_hyperparams {
          activation: RELU_6,
          regularizer {
            l2_regularizer {
              weight: 0.00004
            }
          }
          initializer {
            truncated_normal_initializer {
              stddev: 0.03
              mean: 0.0
            }
          }
        }
      }
    }
    feature_extractor {
      type: 'ssd_inception_v2'
      min_depth: 16
      depth_multiplier: 1.0
      conv_hyperparams {
        activation: RELU_6,
        regularizer {
          l2_regularizer {
            weight: 0.00004
          }
        }
        initializer {
          truncated_normal_initializer {
            stddev: 0.03
            mean: 0.0
          }
        }
        batch_norm {
          train: true,
          scale: true,
          center: true,
          decay: 0.9997,
          epsilon: 0.001,
        }
      }
      override_base_feature_extractor_hyperparams: true
    }
    loss {
      classification_loss {
        weighted_sigmoid {
        }
      }
      localization_loss {
        weighted_smooth_l1 {
        }
      }
      hard_example_miner {
        num_hard_examples: 3000
        iou_threshold: 0.99
        loss_type: CLASSIFICATION
        max_negatives_per_positive: 3
        min_negatives_per_image: 0
      }
      classification_weight: 1.0
      localization_weight: 1.0
    }
    normalize_loss_by_num_matches: true
    post_processing {
      batch_non_max_suppression {
        score_threshold: 1e-8
        iou_threshold: 0.6
        max_detections_per_class: 100
        max_total_detections: 100
      }
      score_converter: SIGMOID
    }
  }
}

train_config: {
  batch_size: 12
  optimizer {
    rms_prop_optimizer: {
      learning_rate: {
        exponential_decay_learning_rate {
          initial_learning_rate: 0.004
          decay_steps: 800720
          decay_factor: 0.95
        }
      }
      momentum_optimizer_value: 0.9
      decay: 0.9
      epsilon: 1.0
    }
  }
  fine_tune_checkpoint: "/home/gpuaffine/PNG/Mukesh/Tensorflow/models/research/pretrained_model/model.ckpt"
  from_detection_checkpoint: true
  # Note: The below line limits the training process to 200K steps, which we
  # empirically found to be sufficient enough to train the pets dataset. This
  # effectively bypasses the learning rate schedule (the learning rate will
  # never decay). Remove the below line to train indefinitely.
  num_steps: 10000
  data_augmentation_options {
    random_horizontal_flip {
    }
  }
  data_augmentation_options {
    ssd_random_crop {
    }
  }
}

train_input_reader: {
  tf_record_input_reader {
    input_path: "/home/gpuaffine/PNG/Dristi/train.record"
  }
  label_map_path: "/home/gpuaffine/PNG/Dristi/object-detection.pbtxt"
}

eval_config: {
  num_examples: 160
  # the number of images to disply in Tensorboard while training
  num_visualizations: 160
  # Note: The below line limits the evaluation process to 10 evaluations.
  # Remove the below line to evaluate indefinitely.
#   max_evals: 10
}

eval_input_reader: {
  tf_record_input_reader {
    input_path: "/home/gpuaffine/PNG/Dristi/val.record"
  }
  label_map_path: "/home/gpuaffine/PNG/Dristi/object-detection.pbtxt"
  shuffle: false
  num_readers: 1
}

Overwriting ./ssd_inception_v2_coco.config


In [22]:
# Specifying the directory name for saving the model
model_dir = 'training/'
if selected_model.startswith("ssd_custom_model"):
    DOWNLOAD_BASE = '/home/gpuaffine/PNG/Dristi/'
    cmd = 'cp -avr'+DOWNLOAD_BASE + model_dir+' .'
    os.system(cmd)
else:
    # Optionally remove content in output model directory to fresh start.
    !rm -rf {model_dir}
    os.makedirs(model_dir, exist_ok=True)

In [23]:
# # Specifying the directory name for saving the model - Run this if you want the custom model to be used as initial weights
# And not start from the last checkpoint in a previous training folder
# This is normally used when we want to use a checkpoint which is not the last checkpoint of a training session 
# as our starting checkpoint for further training
# Uncomment the below codes if that is the case
# model_dir = 'training/'
# # Optionally remove content in output model directory to fresh start.
# !rm -rf {model_dir}
# os.makedirs(model_dir, exist_ok=True)

In [ ]:
# Training the model through transferred learning
!python /home/gpuaffine/PNG/Mukesh/Tensorflow/models/research/object_detection/model_main.py \
    --pipeline_config_path={pipeline_fname} \
    --model_dir={model_dir} \
    --alsologtostderr \
    --num_train_steps={num_steps} \
    --num_eval_steps={num_eval_steps}

W0429 15:50:23.956998 140382354622208 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

W0429 15:50:24.509033 140382354622208 module_wrapper.py:139] From /home/gpuaffine/PNG/Mukesh/Tensorflow/models/research/object_detection/model_main.py:109: The name tf.app.run is deprecated. Please use tf.compat.v1.app.run instead.

W0429 15:50:24.513033 140382354622208 module_wrapper.py:139] From /home/gpuaffine/PNG/Mukesh/Tensorflow/models/research/object_detection/utils/config_util.py:102: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.

W0429 15:50:24.513033 140382354622208 module_wrapper.py:139] From /home/gpuaffine/PNG/Mukesh/Tensorflow/

In [0]:
# Check if the model file is created or not
!ls {model_dir}

###  Move the training folder from current working directory to a separate folder

In [30]:
!pwd

/content/models/research


In [0]:
!mv -avr {model_dir} /home/gpuaffine/PNG/Dristi/

In [ ]:
model_dir = os.path.join('/home/gpuaffine/PNG/Dristi/',model_dir)

## Exporting a Trained Inference Graph
Once your training job is complete, you need to extract the newly trained inference graph, which will be later used to perform the object detection

In [36]:
import re
import numpy as np

output_directory = '/home/gpuaffine/PNG/Dristi/fine_tuned_modelv1'

lst = os.listdir(model_dir)
lst = [l for l in lst if 'model.ckpt-' in l and '.meta' in l]
steps=np.array([int(re.findall('\d+', l)[0]) for l in lst])
last_model = lst[steps.argmax()].replace('.meta', '')
# Uncomment this if you need any other model checkpoint than the last
# steps=[int(re.findall('\d+', l)[0]) for l in lst]
# last_model = lst[steps.index(13178)].replace('.meta', '')

last_model_path = os.path.join(model_dir, last_model)
print(last_model_path)
!python /content/models/research/object_detection/export_inference_graph.py \
    --input_type=image_tensor \
    --pipeline_config_path={pipeline_fname} \
    --output_directory={output_directory} \
    --trained_checkpoint_prefix={last_model_path}

training3/model.ckpt-20000
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



W0408 19:43:00.416541 139884751259520 module_wrapper.py:139] From /content/models/research/object_detection/export_inference_graph.py:145: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.


W0408 19:43:00.422029 139884751259520 module_wrapper.py:139] From /content/models/research/object_detection/exporter.py:402: The name tf.gfile.MakeDirs is deprecated. Please use tf.io.gfile.makedirs instead.


W0408 19:43:00.422313 139884751259520 module_wrapper.py:139] From /content/models/research/object_detection/exporter.py:121: The name tf.placeholder is deprecated. Please use tf.c

In [38]:
# Displaying the content of the fine tune model output directory
!ls {output_directory}

checkpoint			model.ckpt.index  saved_model
frozen_inference_graph.pb	model.ckpt.meta
model.ckpt.data-00000-of-00001	pipeline.config


In [0]:
# Preserving the fine tuned checkpoints for later use
# tar the output directory
tar_file_name = output_directory.split('/')[-1]+'.tar.gz'
def tardir(path, tar_name):
    with tarfile.open(tar_name, "w:gz") as tar_handle:
        for root, dirs, files in os.walk(path):
            for file in files:
                tar_handle.add(os.path.join(root, file))
tardir(output_directory, tar_file_name)
tar.close()

## Checking the model .pb file

In [0]:
import os

pb_fname = os.path.join(os.path.abspath(output_directory), "frozen_inference_graph.pb")
assert os.path.isfile(pb_fname), '`{}` not exist'.format(pb_fname)

In [43]:
!ls -alh {pb_fname}

-rw-r--r-- 1 root root 54M Apr  8 19:43 /content/models/research/fine_tuned_modelv3/frozen_inference_graph.pb


## Evaluation of the model

### Evaluation on the validation dataset

In [0]:
# Specifying the directory name for saving the model
eval_dir = 'eval_val/'
# Optionally remove content in output model directory to fresh start.
!rm -rf {eval_dir}
os.makedirs(eval_dir, exist_ok=True)

In [48]:
# Evaluating the trained model
# The evaluations will be stored in 'eval' folder
# We can then view the evaluations using tensorboard
!python /content/models/research/object_detection/legacy/eval.py \
    --logtostderr \
    --run_once \
    --pipeline_config_path={pipeline_fname} \
    --checkpoint_dir={model_dir} \
    --eval_dir={eval_dir}

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.




Instructions for updating:
Use object_detection/model_main.py.
W0408 20:02:27.224776 140048150542208 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/absl/app.py:250: main (from __main__) is deprecated and will be removed in a future version.
Instructions for updating:
Use object_detection/model_main.py.

W0408 20:02:27.224973 140048150542208 module_wrapper.py:139] From /content/models/research/object_detection/legacy/eval.py:88: The name tf.gfile.MakeDirs is deprecated. Please use tf.io.gfile.makedirs instead.


W0408 20:02:27.225277 140048150542208 module_wrapper.py:139] From /content/models/research/object_d

### Evaluation on the train dataset

In [0]:
# Modifying the the content of the model config file - re-configuration - making the train data as the evaluation dataset

num_examples = 490
with open(pipeline_fname) as f:
    s = f.read()
with open(pipeline_fname, 'w') as f:
    # tfrecord files train and test.
    s = re.sub(
        '(input_path: ".*?)(val.record)(.*?")', 'input_path: "{}"'.format(train_record_fname), s)
    # Set number of examples.
    s = re.sub('num_examples: [0-9]+',
               'num_examples: {}'.format(num_examples), s)
    f.write(s)

In [50]:
# Displaying the content of the model config file
!cat {pipeline_fname}

# SSD with Inception v2 configuration for MSCOCO Dataset.
# Users should configure the fine_tune_checkpoint field in the train config as
# well as the label_map_path and input_path fields in the train_input_reader and
# eval_input_reader. Search for "PATH_TO_BE_CONFIGURED" to find the fields that
# should be configured.

model {
  ssd {
    num_classes: 6
    box_coder {
      faster_rcnn_box_coder {
        y_scale: 10.0
        x_scale: 10.0
        height_scale: 5.0
        width_scale: 5.0
      }
    }
    matcher {
      argmax_matcher {
        matched_threshold: 0.5
        unmatched_threshold: 0.5
        ignore_thresholds: false
        negatives_lower_than_unmatched: true
        force_match_for_each_row: true
      }
    }
    similarity_calculator {
      iou_similarity {
      }
    }
    anchor_generator {
      ssd_anchor_generator {
        num_layers: 6
        min_scale: 0.2
        max_scale: 0.95
        aspect_ratios: 1.0
        aspect_ratios: 2.0
        aspect_

In [51]:
#editing the configuration file to add the path for the TFRecords files, pbtxt,batch_size,num_steps,num_classes.
# any image augmentation, hyperparemeter tunning (drop out, batch normalization... etc) would be editted here

%%writefile {pipeline_fname}
# SSD with Inception v2 configuration for MSCOCO Dataset.
# Users should configure the fine_tune_checkpoint field in the train config as
# well as the label_map_path and input_path fields in the train_input_reader and
# eval_input_reader. Search for "PATH_TO_BE_CONFIGURED" to find the fields that
# should be configured.

model {
  ssd {
    num_classes: 6
    box_coder {
      faster_rcnn_box_coder {
        y_scale: 10.0
        x_scale: 10.0
        height_scale: 5.0
        width_scale: 5.0
      }
    }
    matcher {
      argmax_matcher {
        matched_threshold: 0.5
        unmatched_threshold: 0.5
        ignore_thresholds: false
        negatives_lower_than_unmatched: true
        force_match_for_each_row: true
      }
    }
    similarity_calculator {
      iou_similarity {
      }
    }
    anchor_generator {
      ssd_anchor_generator {
        num_layers: 6
        min_scale: 0.2
        max_scale: 0.95
        aspect_ratios: 1.0
        aspect_ratios: 2.0
        aspect_ratios: 0.5
        aspect_ratios: 3.0
        aspect_ratios: 0.3333
        reduce_boxes_in_lowest_layer: true
      }
    }
    image_resizer {
      fixed_shape_resizer {
        height: 300
        width: 300
      }
    }
    box_predictor {
      convolutional_box_predictor {
        min_depth: 0
        max_depth: 0
        num_layers_before_predictor: 0
        use_dropout: false
        dropout_keep_probability: 0.8
        kernel_size: 3
        box_code_size: 4
        apply_sigmoid_to_scores: false
        conv_hyperparams {
          activation: RELU_6,
          regularizer {
            l2_regularizer {
              weight: 0.00004
            }
          }
          initializer {
            truncated_normal_initializer {
              stddev: 0.03
              mean: 0.0
            }
          }
        }
      }
    }
    feature_extractor {
      type: 'ssd_inception_v2'
      min_depth: 16
      depth_multiplier: 1.0
      conv_hyperparams {
        activation: RELU_6,
        regularizer {
          l2_regularizer {
            weight: 0.00004
          }
        }
        initializer {
          truncated_normal_initializer {
            stddev: 0.03
            mean: 0.0
          }
        }
        batch_norm {
          train: true,
          scale: true,
          center: true,
          decay: 0.9997,
          epsilon: 0.001,
        }
      }
      override_base_feature_extractor_hyperparams: true
    }
    loss {
      classification_loss {
        weighted_sigmoid {
        }
      }
      localization_loss {
        weighted_smooth_l1 {
        }
      }
      hard_example_miner {
        num_hard_examples: 3000
        iou_threshold: 0.99
        loss_type: CLASSIFICATION
        max_negatives_per_positive: 3
        min_negatives_per_image: 0
      }
      classification_weight: 1.0
      localization_weight: 1.0
    }
    normalize_loss_by_num_matches: true
    post_processing {
      batch_non_max_suppression {
        score_threshold: 1e-8
        iou_threshold: 0.6
        max_detections_per_class: 100
        max_total_detections: 100
      }
      score_converter: SIGMOID
    }
  }
}

train_config: {
  batch_size: 12
  optimizer {
    rms_prop_optimizer: {
      learning_rate: {
        exponential_decay_learning_rate {
          initial_learning_rate: 0.004
          decay_steps: 800720
          decay_factor: 0.95
        }
      }
      momentum_optimizer_value: 0.9
      decay: 0.9
      epsilon: 1.0
    }
  }
  fine_tune_checkpoint: "/content/models/research/pretrained_model/model.ckpt"
  from_detection_checkpoint: true
  # Note: The below line limits the training process to 200K steps, which we
  # empirically found to be sufficient enough to train the pets dataset. This
  # effectively bypasses the learning rate schedule (the learning rate will
  # never decay). Remove the below line to train indefinitely.
  num_steps: 20000
  data_augmentation_options {
    random_horizontal_flip {
    }
  }
  data_augmentation_options {
    ssd_random_crop {
    }
  }
}

train_input_reader: {
  tf_record_input_reader {
    input_path: "train.record"
  }
  label_map_path: "object-detection.pbtxt"
}

eval_config: {
  num_examples: 120
  # the number of images to disply in Tensorboard while training
  num_visualizations: 120
  # Note: The below line limits the evaluation process to 10 evaluations.
  # Remove the below line to evaluate indefinitely.
  # max_evals: 10
}

eval_input_reader: {
  tf_record_input_reader {
    input_path: "train.record"
  }
  label_map_path: "object-detection.pbtxt"
  shuffle: false
  num_readers: 1
}

Overwriting /content/models/research/object_detection/samples/configs/ssd_inception_v2_coco.config


In [0]:
# Specifying the directory name for saving the model
eval_dir = 'eval_train/'
# Optionally remove content in output model directory to fresh start.
!rm -rf {eval_dir}
os.makedirs(eval_dir, exist_ok=True)

In [53]:
# Evaluating the trained model
# The evaluations will be stored in 'eval' folder
# We can then view the evaluations using tensorboard
!python /content/models/research/object_detection/model_main.py \
    --logtostderr \
    --run_once \
    --pipeline_config_path={pipeline_fname} \
    --checkpoint_dir={model_dir} \
    --eval_dir={eval_dir}

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



W0408 20:09:43.351142 140711631726464 module_wrapper.py:139] From /content/models/research/object_detection/utils/config_util.py:102: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.


W0408 20:09:43.354367 140711631726464 module_wrapper.py:139] From /content/models/research/object_detection/model_lib.py:628: The name tf.logging.warning is deprecated. Please use tf.compat.v1.logging.warning instead.

W0408 20:09:43.354510 140711631726464 model_lib.py:629] Forced number of epochs for all eval validations to be 1.

W0408 20:09:43.354607 140711631726464 module_wrapper.py:139] From /content/models/resea

In [0]:
# Specifying the directory name for saving the model
eval_dir = 'eval_train_pascal/'
# Optionally remove content in output model directory to fresh start.
!rm -rf {eval_dir}
os.makedirs(eval_dir, exist_ok=True)

In [55]:
# Evaluating the trained model
# The evaluations will be stored in 'eval' folder
# We can then view the evaluations using tensorboard
!python /content/models/research/object_detection/legacy/eval.py \
    --logtostderr \
    --run_once \
    --pipeline_config_path={pipeline_fname} \
    --checkpoint_dir={model_dir} \
    --eval_dir={eval_dir}

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.




Instructions for updating:
Use object_detection/model_main.py.
W0408 20:10:22.438330 140237415835520 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/absl/app.py:250: main (from __main__) is deprecated and will be removed in a future version.
Instructions for updating:
Use object_detection/model_main.py.

W0408 20:10:22.438514 140237415835520 module_wrapper.py:139] From /content/models/research/object_detection/legacy/eval.py:88: The name tf.gfile.MakeDirs is deprecated. Please use tf.io.gfile.makedirs instead.


W0408 20:10:22.438776 140237415835520 module_wrapper.py:139] From /content/models/research/object_d

In [0]:
!ls

a3c_blogpost			  lm_commonsense
adversarial_crypto		  lstm_object_detection
adversarial_logit_pairing	  marco
adversarial_text		  maskgan
adv_imagenet_models		  namignizer
attention_ocr			  neural_gpu
audioset			  neural_programmer
autoaugment			  next_frame_prediction
autoencoder			  nst_blogpost
brain_coder			  object_detection
cognitive_mapping_and_planning	  object-detection.pbtxt
cognitive_planning		  pcl_rl
compression			  pretrained_model
cvt_text			  ptn
deep_contextual_bandits		  qa_kg
deeplab				  README.md
deep_speech			  real_nvp
delf				  rebar
domain_adaptation		  sentiment_analysis
efficient-hrl			  seq2species
eval_train			  setup.py
eval_train_pascal		  skip_thoughts
eval_val			  slim
feelvos				  steve
fine_tuned_model		  street
fivo				  struct2depth
global_objectives		  swivel
im2txt				  tcn
inception			  test.record
keypointnet			  textsum
learned_optimizer		  training
learning_to_remember_rare_events  training.zip
learning_unsupervised_learning	  train.record
lex

## Downloading the train and test evaluations to check in tensorboard - for colab only

In [0]:
# zipping and downloading the evaluations for visualizing in tensorboard
!zip -r training.zip training

updating: training/ (stored 0%)
updating: training/events.out.tfevents.1585799753.fa8e63f00b4d (deflated 94%)
updating: training/model.ckpt-18313.data-00000-of-00001 (deflated 7%)
updating: training/model.ckpt-20000.meta (deflated 7%)
updating: training/model.ckpt-20000.index (deflated 73%)
updating: training/checkpoint (deflated 72%)
updating: training/model.ckpt-17407.meta (deflated 94%)
updating: training/model.ckpt-19220.meta (deflated 94%)
updating: training/model.ckpt-18313.meta (deflated 94%)
updating: training/eval_0/ (stored 0%)
updating: training/eval_0/events.out.tfevents.1585800398.fa8e63f00b4d (deflated 17%)
updating: training/export/ (stored 0%)
updating: training/export/Servo/ (stored 0%)
updating: training/export/Servo/1585812908/ (stored 0%)
updating: training/export/Servo/1585812908/variables/ (stored 0%)
updating: training/export/Servo/1585812908/variables/variables.data-00000-of-00001 (deflated 7%)
updating: training/export/Servo/1585812908/variables/variables.index

In [0]:
files.download("training.zip")

----------------------------------------
Exception happened during processing of request from ('::ffff:127.0.0.1', 56786, 0, 0)
Traceback (most recent call last):
  File "/usr/lib/python3.6/socketserver.py", line 320, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/usr/lib/python3.6/socketserver.py", line 351, in process_request
    self.finish_request(request, client_address)
  File "/usr/lib/python3.6/socketserver.py", line 364, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/usr/lib/python3.6/socketserver.py", line 724, in __init__
    self.handle()
  File "/usr/lib/python3.6/http/server.py", line 418, in handle
    self.handle_one_request()
  File "/usr/lib/python3.6/http/server.py", line 406, in handle_one_request
    method()
  File "/usr/lib/python3.6/http/server.py", line 639, in do_GET
    self.copyfile(f, self.wfile)
  File "/usr/lib/python3.6/http/server.py", line 800, in copyfile
    shutil.copyfil

In [0]:
!zip -r eval_val.zip eval_val

  adding: eval_val/ (stored 0%)
  adding: eval_val/events.out.tfevents.1585814209.fa8e63f00b4d (deflated 25%)
  adding: eval_val/pipeline.config (deflated 65%)


In [0]:
files.download("eval_val.zip")

In [0]:
!zip -r eval_train.zip eval_train

updating: eval_train/ (stored 0%)


In [0]:
files.download("eval_train.zip")

In [0]:
!zip -r eval_train_pascal.zip eval_train_pascal

  adding: eval_train_pascal/ (stored 0%)
  adding: eval_train_pascal/events.out.tfevents.1585814690.fa8e63f00b4d (deflated 5%)
  adding: eval_train_pascal/pipeline.config (deflated 65%)


In [0]:
files.download("eval_train_pascal.zip")

In [0]:
## ------end of code